In [1]:
import os
print(os.getcwd())

/Users/mikolajmazur/Documents/programming/Python/python-random/system-scripts


### 1. Wczytaj 3 pliki CSV z logami

In [2]:
import pandas as pd
from pathlib import Path
from datetime import timedelta

linux_path = Path("../example-files/linux_log.csv")
windows_path = Path("../example-files/windows_log.csv")
web_path = Path("../example-files/web_log.csv")

linux_logs = pd.read_csv(linux_path, parse_dates=['timestamp'])
windows_logs = pd.read_csv(windows_path, parse_dates=['timestamp'])
web_logs = pd.read_csv(web_path, parse_dates=['timestamp'])

# data unification
# i dont unifie data to see how more challanging quering will be

In [3]:
linux_logs.head()

,timestamp,IP,user,status,process
0,2025-09-28 09:00:10,192.168.0.5,root,FAILED,sshd
1,2025-09-28 09:00:40,192.168.0.6,admin,FAILED,sshd
2,2025-09-28 09:01:00,192.168.0.8,guest,OK,cron
3,2025-09-28 09:01:30,192.168.0.5,root,OK,sudo
4,2025-09-28 09:02:00,10.0.0.3,anna,FAILED,sshd


In [4]:
windows_logs.head()

,timestamp,IP,username,event,application
0,2025-09-28 09:00:11,192.168.0.5,Administrator,LoginFailed,RemoteDesktop
1,2025-09-28 09:00:43,192.168.0.6,Admin,LoginFailed,RemoteDesktop
2,2025-09-28 09:01:03,192.168.0.8,Guest,LoginSuccess,Explorer
3,2025-09-28 09:01:33,192.168.0.5,Administrator,LoginSuccess,RemoteDesktop
4,2025-09-28 09:02:05,10.0.0.3,Anna,LoginFailed,RemoteDesktop


In [5]:
web_logs.head()

,timestamp,IP,user_web,endpoint,status_code
0,2025-09-28 09:00:15,192.168.0.5,root,/login,401
1,2025-09-28 09:00:47,192.168.0.6,admin,/login,401
2,2025-09-28 09:01:05,192.168.0.8,guest,/login,200
3,2025-09-28 09:01:35,192.168.0.5,root,/data,200
4,2025-09-28 09:02:10,10.0.0.3,anna,/login,401


### 2. Znajdź wspólne adresy IP występujące we wszystkich logach.

In [6]:
common_ips = set(linux_logs['IP']).intersection(set(windows_logs['IP'])).intersection(set(web_logs['IP']))
common_ips

{'10.0.0.3', '192.168.0.5', '192.168.0.6', '192.168.0.8'}

### 3. Używając merge_asof() połącz linux_log i windows_log

In [7]:
merged = pd.merge_asof(
    linux_logs.sort_values(['timestamp']),
    windows_logs.sort_values(['timestamp']),
    on=['timestamp'],
    by=['IP'],
    direction='nearest',
    tolerance=pd.Timedelta('10s')
)

merged

,timestamp,IP,user,status,process,username,event,application
0,2025-09-28 09:00:10,192.168.0.5,root,FAILED,sshd,Administrator,LoginFailed,RemoteDesktop
1,2025-09-28 09:00:40,192.168.0.6,admin,FAILED,sshd,Admin,LoginFailed,RemoteDesktop
2,2025-09-28 09:01:00,192.168.0.8,guest,OK,cron,Guest,LoginSuccess,Explorer
3,2025-09-28 09:01:30,192.168.0.5,root,OK,sudo,Administrator,LoginSuccess,RemoteDesktop
4,2025-09-28 09:02:00,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop
5,2025-09-28 09:02:15,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop
6,2025-09-28 09:02:30,192.168.0.6,admin,OK,sshd,Admin,LoginSuccess,RemoteDesktop
7,2025-09-28 09:02:45,192.168.0.5,root,FAILED,sshd,NaN,NaN,NaN
8,2025-09-28 09:03:00,192.168.0.8,guest,OK,ftp,Guest,LoginFailed,SMB


### 4. Dołącz dane z web_log w ten sam sposób

In [8]:
merged = pd.merge_asof(
    merged.sort_values(['timestamp']),
    web_logs.sort_values(['timestamp']),
    on=['timestamp'],
    by=['IP'],
    direction='nearest',
    tolerance=pd.Timedelta('10s')
)

merged

,timestamp,IP,user,status,process,username,event,application,user_web,endpoint,status_code
0,2025-09-28 09:00:10,192.168.0.5,root,FAILED,sshd,Administrator,LoginFailed,RemoteDesktop,root,/login,401.0
1,2025-09-28 09:00:40,192.168.0.6,admin,FAILED,sshd,Admin,LoginFailed,RemoteDesktop,admin,/login,401.0
2,2025-09-28 09:01:00,192.168.0.8,guest,OK,cron,Guest,LoginSuccess,Explorer,guest,/login,200.0
3,2025-09-28 09:01:30,192.168.0.5,root,OK,sudo,Administrator,LoginSuccess,RemoteDesktop,root,/data,200.0
4,2025-09-28 09:02:00,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop,anna,/login,401.0
5,2025-09-28 09:02:15,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop,anna,/login,401.0
6,2025-09-28 09:02:30,192.168.0.6,admin,OK,sshd,Admin,LoginSuccess,RemoteDesktop,NaN,NaN,NaN
7,2025-09-28 09:02:45,192.168.0.5,root,FAILED,sshd,NaN,NaN,NaN,NaN,NaN,NaN
8,2025-09-28 09:03:00,192.168.0.8,guest,OK,ftp,Guest,LoginFailed,SMB,guest,/login,401.0


### 5. Wyświetl wynikowe korelacje – czy dane IP generują błędy w więcej niż jednym systemie?

In [9]:
merged[((merged['status'] == 'FAILED') & (merged['event'] == 'LoginFailed')) | ((merged['status'] == 'FAILED') & (merged['event'] == 'LoginFailed') & (merged['status_code'] == 401.0))]

,timestamp,IP,user,status,process,username,event,application,user_web,endpoint,status_code
0,2025-09-28 09:00:10,192.168.0.5,root,FAILED,sshd,Administrator,LoginFailed,RemoteDesktop,root,/login,401.0
1,2025-09-28 09:00:40,192.168.0.6,admin,FAILED,sshd,Admin,LoginFailed,RemoteDesktop,admin,/login,401.0
4,2025-09-28 09:02:00,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop,anna,/login,401.0
5,2025-09-28 09:02:15,10.0.0.3,anna,FAILED,sshd,Anna,LoginFailed,RemoteDesktop,anna,/login,401.0


### 6. Dla procesu 'sshd' i adresu IP= 192.168.0.6 w linux_log powodującego błędy, sprawdź czy to samo IP generuje błędy logowania w windows_log i web_log w przedziale czsu +-30 sekund

In [10]:
target_ip = '192.168.0.6'
target_log = linux_logs[(linux_logs["IP"] == target_ip) &
    (linux_logs["process"] == "sshd") &
    (linux_logs["status"].astype(str).str.upper() == "FAILED")].sort_values(['timestamp'])

windows_ip = windows_logs[windows_logs["IP"] == target_ip].sort_values("timestamp")
web_ip = web_logs[web_logs["IP"] == target_ip].sort_values("timestamp")

for _, row in target_log.iterrows():
    ts = row["timestamp"]

    win_match = windows_ip[
        (windows_ip["event"] == "LoginFailed") &
        (windows_ip["timestamp"].between(ts - timedelta(seconds=30), ts + timedelta(seconds=30)))
    ]

    web_match = web_ip[
        (web_ip["status_code"].astype(float) != 200.0) &
        (web_ip["timestamp"].between(ts - timedelta(seconds=30), ts + timedelta(seconds=30)))
    ]

    print("Linux FAILED login")
    print(row[["timestamp", "IP", "user", "process"]])
    print(f"-> Windows match within ±30s: {not win_match.empty}")
    if not win_match.empty:
        print(win_match[["timestamp", "username", "event", "application"]])
    print(f"-> Web match within ±30s: {not web_match.empty}")
    if not web_match.empty:
        print(web_match[["timestamp", "user_web", "endpoint", "status_code"]])
    print()

Linux FAILED login
timestamp    2025-09-28 09:00:40
IP                   192.168.0.6
user                       admin
process                     sshd
Name: 1, dtype: object
-> Windows match within ±30s: True
            timestamp username        event    application
1 2025-09-28 09:00:43    Admin  LoginFailed  RemoteDesktop
-> Web match within ±30s: True
            timestamp user_web endpoint  status_code
1 2025-09-28 09:00:47    admin   /login          401

